## Heroes of Pymoli Data Analysis

In [2]:
# Dependencies and set up
import pandas as pd

filePath = 'Resources/purchase_data.csv'

purchase_data = pd.read_csv(filePath)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count
To get the total number of players, we need to count the total number of unique Screen Names in column "SN". The result is 576 unique players

In [3]:
len(purchase_data["SN"].unique())

576

### Purchasing Analysis (Total)
* Number of unique items: 183
* Average Price
* Number of Purchases
* Total Revenue

In [4]:
# pd.options.display.float_format = '${:,.2f}'.format # format floats to currency formatting
purchase_data["Price"].mean() # 3.05
len(purchase_data["Purchase ID"].unique()) #780
purchase_data["Price"].sum() # 2379.77

# Create dictionary to store summary values
summary_dict = {
    "Number of Unique Items": int(len(purchase_data["Item ID"].unique())),
    "Average Price": "$" + str(round(purchase_data["Price"].mean(),2)),
    "Number of Purchases": int(len(purchase_data["Purchase ID"].unique())),
    "Total Revenue": "$" + str(purchase_data["Price"].sum())
}

summary = pd.DataFrame(summary_dict,[0])
summary.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


### Gender Demographics
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [5]:
purchase_data.head()
df_deduped = purchase_data.drop_duplicates(subset='SN', keep='first') # remove duplicates from "SN" colume in order to get counts of just the unique players

count_male = df_deduped["Gender"].value_counts()["Male"]
count_female = df_deduped["Gender"].value_counts()["Female"]
count_other = df_deduped["Gender"].value_counts()["Other / Non-Disclosed"]
count_total = df_deduped["Gender"].count()
male_pct = round((count_male / count_total) * 100,2)
female_pct = round((count_female / count_total) * 100,2)
other_pct = round((count_other / count_total) * 100,2)


gender_demo_summary_dict = {
    "Male": df_deduped["Gender"].value_counts()["Male"],
    "Female": df_deduped["Gender"].value_counts()["Female"],
    "Male": df_deduped["Gender"].value_counts()["Male"],
}

gender_summary = pd.DataFrame({
    "Total Count":[count_male,count_female,count_other],
    "Percentage of Playes":[male_pct,female_pct,other_pct]},
    index=['Male','Female',"Other / Non-Disclosed"])
gender_summary

,Total Count,Percentage of Playes
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


### Purchasing Analysis (Gender)
* Purchase Count
* Average Purchase Price
* Total Purchase Value
* Average Purchase Total per Person by Gender

In [31]:
# Group Purchase Data by Gender
gender_groups = purchase_data.groupby("Gender")

# Calculate Fields
avg_price = gender_groups.mean()['Price'] # Average Purchase Price
tot_purchase = gender_groups.sum()['Price'] # Total Purchase Value
num_purchases = gender_groups.count()['Purchase ID'] # Purchase Count
avg_purchase_per_person = tot_purchase / df_deduped["Gender"].value_counts()

# Create Summary Dataframe
purchasing_analysis_gender = pd.DataFrame({
    'Purchase Count':num_purchases,
    'Average Purchase Price':avg_price,
    'Total Purchase Value':tot_purchase,
    'Avg Total Purchase per Person':avg_purchase_per_person
})

# Formatting
purchasing_analysis_gender['Average Purchase Price'] = purchasing_analysis_gender['Average Purchase Price'].map("${:.2f}".format)
purchasing_analysis_gender['Total Purchase Value'] = purchasing_analysis_gender['Total Purchase Value'].map("${:,.2f}".format)
purchasing_analysis_gender['Avg Total Purchase per Person'] = purchasing_analysis_gender['Avg Total Purchase per Person'].map("${:.2f}".format)
purchasing_analysis_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics
* Percentage and Count of Players by Age Group

In [67]:
# Get stats for age
purchase_data['Age'].describe() # min: 7, max: 45

# Create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
bin_names = ['0-10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add column, Age Group, using bins
purchase_data['Age Group'] = pd.cut(purchase_data['Age'].loc[:], bins, labels=bin_names)
df_deduped['Age Group'] = pd.cut(purchase_data['Age'].loc[:], bins, labels=bin_names)

count_total = df_deduped.groupby('Age Group').count()['SN']
pct_of_total = round((count_total / count_total.sum()) * 100,2)
pct_of_total

age_demo_summary = pd.DataFrame({
    'Total Count':count_total,
    'Percentage of Players':pct_of_total
})
age_demo_summary

/Users/gfidanli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Total Count,Percentage of Players
Age Group,,
0-10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


### Purchasing Analysis (Age)
* Purchase Count
* Average Purchase Price
* Total Purchase Value
* Average Purchase Total per Person by Age Group

In [57]:
# Group purhase data by age bins
by_age_group = purchase_data.groupby('Age Group')

# Calculate Metrics
avg_price = by_age_group.mean()['Price'] # Average Purchase Price
tot_purchase = by_age_group.sum()['Price'] # Total Purchase Value
num_purchases = by_age_group.count()['Purchase ID'] # Purchase Count
avg_purchase_per_person = tot_purchase / df_deduped["Age Group"].value_counts()

# Create Summary Dataframe
purch_analysis_age = pd.DataFrame({
    'Purchase Count':num_purchases,
    'Average Purchase Price':avg_price,
    'Total Purchase Value':tot_purchase,
    'Avg Total Purchase per Person':avg_purchase_per_person
})

# Formatting
purch_analysis_age['Average Purchase Price'] = purch_analysis_age['Average Purchase Price'].map(" ${:.2f}".format)
purch_analysis_age['Total Purchase Value'] = purch_analysis_age['Total Purchase Value'].map(" ${:,.2f}".format)
purch_analysis_age['Avg Total Purchase per Person'] = purch_analysis_age['Avg Total Purchase per Person'].map(" ${:.2f}".format)
purch_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0-10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Top Spenders
* Identify the top 5 spenders in the game by total purchase value, then list (in a table)
 * SN
 * Purchase Count
 * Average Purchase Price
 * Total Purchase Value

In [93]:
# Group purchase data by player
by_player = purchase_data.groupby('SN')

# Calculate Metrics
num_purchases = by_player['Purchase ID'].count()
avg_price = by_player['Price'].mean()
tot_purchase = by_player['Price'].sum()

# Create Summary Dataframe
spenders = pd.DataFrame({
    'Purchase Count':num_purchases,
    'Average Purchase Price':avg_price,
    'Total Purchase Value':tot_purchase,
})

# Get top 5 by total purchase value
top_spenders = spenders.sort_values(by='Total Purchase Value', ascending=False).head()

# Formatting
top_spenders['Average Purchase Price'] = spenders['Average Purchase Price'].map(" ${:.2f}".format)
top_spenders['Total Purchase Value'] = spenders['Total Purchase Value'].map(" ${:,.2f}".format)

top_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items
* Identify the most profitable items by total purchase value, then list (in a table)
 * Item ID
 * Item Name
 * Purchase Count
 * Item Price
 * Total Purchase Value

In [98]:
# Group purchase data by Item ID, Item Name
by_item = purchase_data.groupby(['Item ID', 'Item Name'])

# Calculate Metrics
num_purchases = by_item['Purchase ID'].count()
price = by_item['Price'].mean()
tot_purchase = by_item['Price'].sum()

# Create Summary Dataframe
items = pd.DataFrame({
    'Purchase Count':num_purchases,
    'Item Price':price,
    'Total Purchase Value':tot_purchase,
})

# Get top 5 by purchase count
top_items = items.sort_values(by='Purchase Count', ascending=False).head()

# Formatting
top_items['Item Price'] = items['Item Price'].map(" ${:.2f}".format)
top_items['Total Purchase Value'] = items['Total Purchase Value'].map(" ${:,.2f}".format)

top_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
